### Basic Usage

In [1]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
import pandas as pd

catalog = DatasetCatalog()
ds = catalog.get("empresas_10k")

loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

records = loader.load(ds)
print("Filas cargadas:", len(records))
df = pd.DataFrame(records)
df.head(3)


Filas cargadas: 40000


,nit,raz_n_social,supervisor,regi_n,departamento_domicilio,ciudad_domicilio,ciiu,macrosector,ingresos_operacionales,ganancia_p_rdida,total_activos,total_pasivos,total_patrimonio,a_o_de_corte
0,899999068,ECOPETROL S.A,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,$144.82,$33.41,$216.85,$125.81,$91.03,2022
1,900112515,REFINERIA DE CARTAGENA S.A.S,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,1921,MANUFACTURA,$27.86,$2.19,$42.84,$16.48,$26.36,2022
2,830095213,ORGANIZACIÓN TERPEL S.A.,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,4661,COMERCIO,$23.60,$0.33,$7.48,$4.47,$3.01,2022


### Detailed Usage

In [2]:
# Adapters -> fuente de datos
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.metadata import SocrataMetadata

# Loader -> servicio
from ingest.loader import RawDatasetLoader

# Catalog -> catálogo de datasets
from ingest.catalog import DatasetCatalog

csv_adapter=CsvAdapter() # para CSV locales
sct_adapter=SocrataAdapter() # para API Socrata
excel_adapter=ExcelAdapter() # para Excel locales

# inyectar adaptadores al loader
loader = RawDatasetLoader(csv_adapter, sct_adapter, excel_adapter)

# El catalogo sabe donde está cada dataset y como leerlo
# No necesitan reescribir url ni rutas
catalog = DatasetCatalog()


In [3]:
all_ds = catalog.list()
print("Total datasets en catálogo:", len(all_ds))
for dataset in all_ds:
    print("-", dataset.name)

Total datasets en catálogo: 7
- empresas_10k
- pib_municipios
- divipola
- empresas_10k_csv
- ciuu_4ac
- ban_rep
- ciuu_4ac_dane


In [4]:
ds = catalog.get("empresas_10k")
meta = loader.metadata(ds)

if isinstance(meta, SocrataMetadata):
    print("Nombre:", meta.name)
    print("Descripción:", meta.description)
    print("\nColumnas:")
    for col in meta.columns:
        print(f"  - {col.name} ({col.fieldName}): {col.dataTypeName}")
else:
    print("Este origen no soporta metadata estructurada.")

Nombre: 10.000 Empresas mas Grandes del País
Descripción: Reporte de las diez mil empresas más grandes del país, todas las cifras en este reporte se muestran en Billones de Pesos. Lo invitamos a participar en la siguiente encuesta https://forms.office.com/r/1cWttkdHqM

Columnas:
  - NIT (nit): number
  - RAZÓN SOCIAL (raz_n_social): text
  - SUPERVISOR (supervisor): text
  - REGIÓN (regi_n): text
  - DEPARTAMENTO DOMICILIO (departamento_domicilio): text
  - CIUDAD DOMICILIO (ciudad_domicilio): text
  - CIIU (ciiu): number
  - MACROSECTOR (macrosector): text
  - INGRESOS OPERACIONALES (ingresos_operacionales): text
  - GANANCIA (PÉRDIDA) (ganancia_p_rdida): text
  - TOTAL ACTIVOS (total_activos): text
  - TOTAL PASIVOS (total_pasivos): text
  - TOTAL PATRIMONIO (total_patrimonio): text
  - Año de Corte (a_o_de_corte): number


In [5]:
ds_csv = catalog.get("empresas_10k_csv") # CSV local
df_csv = pd.DataFrame(loader.load(ds_csv))
df_csv.head(3)

,NIT,RAZÓN SOCIAL,SUPERVISOR,REGIÓN,DEPARTAMENTO DOMICILIO,CIUDAD DOMICILIO,CIIU,MACROSECTOR,INGRESOS OPERACIONALES,GANANCIA (PÉRDIDA),TOTAL ACTIVOS,TOTAL PASIVOS,TOTAL PATRIMONIO,Año de Corte
0,"899,999,068",ECOPETROL S.A,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,$144.82,$33.41,$216.85,$125.81,$91.03,"2,022"
1,"900,112,515",REFINERIA DE CARTAGENA S.A.S,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,"1,921",MANUFACTURA,$27.86,$2.19,$42.84,$16.48,$26.36,"2,022"
2,"830,095,213",ORGANIZACIÓN TERPEL S.A.,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,"4,661",COMERCIO,$23.60,$0.33,$7.48,$4.47,$3.01,"2,022"


In [6]:
ds_ciiu = catalog.get("ciuu_4ac")
df_ciiu = pd.DataFrame(loader.load(ds_ciiu))
df_ciiu.head(3)

,Sección,División,Grupo,clase,Descripción,col_6,col_7,col_8,col_9,col_10,col_11
0,De acuerdo con las Resoluciones número 2306 de...,None,None,NaN,None,None,None,None,None,None,None
1,CIIU Rev. 4 A.C. (actualización 2022),None,None,NaN,None,None,None,None,None,None,None
2,A,1,11,111.0,"Cultivo de cereales (excepto arroz), legumbres...",None,None,None,None,None,None


In [7]:
ds_ciiu = catalog.get("ciuu_4ac_dane")
df_ciiu = pd.DataFrame(loader.load(ds_ciiu))
df_ciiu.head(3)

,División,Grupo,Clase,Descripción
0,ESTRUCTURA DETALLADA DE LA CLASIFICACIÓN INDUS...,,,
1,,,,
2,SECCIÓN A,,,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P..."
